In [ ]:
!pip3 install cornac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 25.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import random

import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.data import Reader
from cornac.metrics import MAE,RMSE,MSE,FMeasure,Precision,Recall,NDCG,NCRR,MRR,AUC,MAP

In [ ]:
with open ('/content/drive/MyDrive/Colab Notebooks/master-project/dataset/mind_10K.csv') as mind_file:
    mind_ = pd.read_csv(mind_file)
mind_['story'] = mind_['story'].astype('int')

In [ ]:
# pre-process data to initialize ctr model
mind_feedback = mind_.loc[:, ['user_id','item_id','rating']]
# feedback = cornac.data.Dataset.from_uir(mind_feedback.itertuples(index=False))
feedback = mind_feedback.apply(lambda x: tuple(x), axis=1).values.tolist()
text = list(mind_['text'])
item_ids = list(mind_['item_id'])
# Instantiate a TextModality, it makes it convenient to work with text auxiliary information
item_text_modality = TextModality(
    corpus=text,
    ids=item_ids,
    tokenizer=BaseTokenizer(sep=" ", stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)

# Define an evaluation method to split feedback into train and test sets
mind_ratio_split = RatioSplit(
    data=feedback,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=item_text_modality,
    verbose=True,
    seed=123,
    rating_threshold=0.5,
)

rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 272
Number of items = 3942
Number of ratings = 7911
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 234
Number of items = 915
Number of ratings = 1336
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 272
Total items = 3942


/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:361: UserWarning: 89 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:361: UserWarning: 3 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


### k=10

In [ ]:
# Instantiate Conv_mf model
convmf_k10 = cornac.models.ConvMF(
    k=10,
    n_epochs=10,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_k10], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/10


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 62837817.58223 Elapsed: 385.6367s Converge: 6283781758223330756979930717221197937072366503726975811584.000000 
Epoch: 2/10


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 3419814.03839 Elapsed: 385.5711s Converge: 0.945577 
Epoch: 3/10


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 105693.98373 Elapsed: 376.0702s Converge: 0.222229 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.7076 | 0.0127 | 0.0155 | 0.0386 |  0.0174 |  0.0345 |       0.0074 |    0.0803 | 3777.1136 |   0.7335



In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/master-project/conv_mf')

In [ ]:
import diversity_mind
def compute_avg_diversity(model, data, feature):
    if (feature=="category"):
        for i in range(len(set(mind_ratio_split.test_set.user_ids))): # get the average result for all users from training data
            result = []
            r = diversity_mind.Retrieval(model=model,data = data,UIDX = i,TOPK = 50,feature = feature)
            d = diversity_mind.Diversity(r.get_history(), r.get_recy())
            result.append(d.compute())
    else:
        useridx = list(set([int(tuple[1]) for tuple in mind_ratio_split.test_set.uid_map]))
        result = []
        i = 0
        while i <= 200:
            # shuffle
            user_pair = random.sample(useridx, 2)
            r0 = diversity_mind.Retrieval(model=model, data=data, UIDX=user_pair[0],TOPK = 50, feature=feature)
            r1 = diversity_mind.Retrieval(model=model, data=data, UIDX=user_pair[1],TOPK = 50, feature=feature)
            d = diversity_mind.Diversity(r0.get_recy(), r1.get_recy())
            result.append(d.compute())
            i += 1
    return np.mean(result)

In [ ]:
data1 = {"model":"Conv_mf",
        "data":"mind",
        "k":10,
        "n_epochs":10,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.7076,"F1":0.0127,"MAP":0.0155,"MRR":0.0386,"NCRR":0.0174,"NDCG":0.0345,"Precision":0.0074,"Recall":0.0803,
        "category diversity":compute_avg_diversity(convmf_k10, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_k10, mind_, "story")}
print(data1['category diversity'])
print(data1['story diversity'])

0.29247192905736735
0.01035529639060549


### k=50

In [ ]:
# Instantiate Conv_mf model
convmf_k50 = cornac.models.ConvMF(
    k=50,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_k50], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 379513999.58062 Elapsed: 11.0629s Converge: 37951399958062175220629721962271940733507912027173279498240.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 14193273.44013 Elapsed: 10.3507s Converge: 0.962601 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 248459.21306 Elapsed: 10.7814s Converge: 0.438471 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.6193 | 0.0088 | 0.0062 | 0.0156 |  0.0054 |  0.0178 |       0.0052 |    0.0543 |   53.7536 |   0.7100



In [ ]:
data2 = {"model":"Conv_mf",
        "data":"mind",
        "k":50,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.6326,"F1":0.0075,"MAP":0.0071,"MRR":0.0263,"NCRR":0.0085,"NDCG":0.0166,"Precision":0.0045,"Recall":0.0375,
        "category diversity":compute_avg_diversity(convmf_k50, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_k50, mind_, "story")}
print(data2['category diversity'])
print(data2['story diversity'])

0.8008326771683433
0.006183437883283891


### k=100

In [ ]:
# Instantiate Conv_mf model
convmf_k100 = cornac.models.ConvMF(
    k=100,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_k100], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 770401392.39065 Elapsed: 11.0666s Converge: 77040139239064872855530841960050437324702187775120117858304.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 26568889.71202 Elapsed: 10.6419s Converge: 0.965513 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 447729.11838 Elapsed: 10.9832s Converge: 0.169387 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.5883 | 0.0063 | 0.0057 | 0.0141 |  0.0046 |  0.0136 |       0.0038 |    0.0383 |   57.9409 |   0.6314



In [ ]:
data3 = {"model":"Conv_mf",
        "data":"mind",
        "k":100,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5988,"F1":0.0067,"MAP":0.0096,"MRR":0.0277,"NCRR":0.0115,"NDCG":0.0186,"Precision":0.0041,"Recall":0.0351,
        "category diversity":compute_avg_diversity(convmf_k100, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_k100, mind_, "story")}
print(data3['category diversity'])
print(data3['story diversity'])

0.46940804354865584
0.04124357224816788


### k=200

In [ ]:
# Instantiate Conv_mf model
convmf_k200 = cornac.models.ConvMF(
    k=200,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_k200], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 1554025367.08290 Elapsed: 15.3974s Converge: 155402536708290484476186309191458311748106233375022945140736.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 43768815.70176 Elapsed: 15.7859s Converge: 0.971835 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 379901.55299 Elapsed: 14.6385s Converge: 0.463557 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.5247 | 0.0035 | 0.0036 | 0.0091 |  0.0024 |  0.0069 |       0.0021 |    0.0172 |   78.4007 |   0.9459



In [ ]:
data4 = {"model":"Conv_mf",
        "data":"mind",
        "k":200,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5240,"F1":0.0033,"MAP":0.0035,"MRR":0.0094,"NCRR":0.0025,"NDCG":0.0066,"Precision":0.0020,"Recall":0.0161,
        "category diversity":compute_avg_diversity(convmf_k200, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_k200, mind_, "story")}
print(data4['category diversity'])
print(data4['story diversity'])

0.18257364988818273
0.006560904486574628


### k=300

In [ ]:
# Instantiate Conv_mf model
convmf_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2340249161.01926 Elapsed: 22.5958s Converge: 234024916101926225495813262982354016608315088101461215674368.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 59325666.40832 Elapsed: 22.2508s Converge: 0.974650 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 237789.14081 Elapsed: 19.5321s Converge: 0.653630 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.5548 | 0.0038 | 0.0035 | 0.0084 |  0.0022 |  0.0065 |       0.0023 |    0.0166 |  107.3227 |   0.6894



In [ ]:
data5 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5323,"F1":0.0028,"MAP":0.0034,"MRR":0.0100,"NCRR":0.0025,"NDCG":0.0058,"Precision":0.0017,"Recall":0.0148,
        "category diversity":compute_avg_diversity(convmf_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_k300, mind_, "story")}
print(data5['category diversity'])
print(data5['story diversity'])

0.8507972796334449
0.013198428315189702


### k=400

In [ ]:
# Instantiate Conv_mf model
convmf_k400 = cornac.models.ConvMF(
    k=400,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_k400], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 3088016526.95882 Elapsed: 30.2472s Converge: 308801652695881988189288980457039930598834237641864016560128.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 79649441.18781 Elapsed: 30.1294s Converge: 0.974207 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 343869.26323 Elapsed: 29.6780s Converge: 0.708721 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.5388 | 0.0028 | 0.0040 | 0.0149 |  0.0043 |  0.0069 |       0.0016 |    0.0155 |  156.5991 |   0.6887



In [ ]:
data6 = {"model":"Conv_mf",
        "data":"mind",
        "k":400,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5127,"F1":0.0015,"MAP":0.0029,"MRR":0.0044,"NCRR":0.0010,"NDCG":0.0041,"Precision":0.0009,"Recall":0.0148,
        "category diversity":compute_avg_diversity(convmf_k400, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_k400, mind_, "story")}
print(data6['category diversity'])
print(data6['story diversity'])

1.1256427597095529
0.01822516777978173


### k=300, lambda_u=10, lambda_v=100

In [ ]:
# Instantiate Conv_mf model
convmf_u10_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=10,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_u10_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2337977217.16773 Elapsed: 20.2531s Converge: 233797721716772998862943556754415200009852940369584799612928.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 60657428.36784 Elapsed: 22.5090s Converge: 0.974056 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 253307.42520 Elapsed: 19.1539s Converge: 0.679764 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.5936 | 0.0058 | 0.0068 | 0.0213 |  0.0077 |  0.0144 |       0.0034 |    0.0356 |  105.0060 |   0.7113



In [ ]:
data7 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":10,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5495,"F1":0.0042,"MAP":0.0038,"MRR":0.0100,"NCRR":0.0028,"NDCG":0.0077,"Precision":0.0025,"Recall":0.0189,
        "category diversity":compute_avg_diversity(convmf_u10_v100_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_u10_v100_k300, mind_, "story")}
print(data7['category diversity'])
print(data7['story diversity'])

0.7161166988128607
0.008274452692399906


### k=300, lambda_u=100, lambda_v=100

In [ ]:
# Instantiate Conv_mf model
convmf_u100_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=100,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_u100_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2338647361.93649 Elapsed: 23.1225s Converge: 233864736193648511982689479646470093982010575805368468766720.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 61086690.93092 Elapsed: 20.9179s Converge: 0.973879 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 308200.62016 Elapsed: 22.1810s Converge: 0.662720 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.6208 | 0.0079 | 0.0127 | 0.0425 |  0.0183 |  0.0257 |       0.0046 |    0.0507 |  112.3824 |   0.7270



In [ ]:
data8 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":100,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5866,"F1":0.0061,"MAP":0.0053,"MRR":0.0140,"NCRR":0.0048,"NDCG":0.0135,"Precision":0.0036,"Recall":0.0367,
        "category diversity":compute_avg_diversity(convmf_u100_v100_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_u100_v100_k300, mind_, "story")}
print(data8['category diversity'])
print(data8['story diversity'])

0.25739493466368873
0.0017053178693166283


### k=300, lambda_u=1000, lambda_v=100

In [ ]:
# Instantiate Conv_mf model
convmf_u1000_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1000,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)
# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_u1000_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2338354956.14641 Elapsed: 22.6533s Converge: 233835495614641317580473415262422995715055416475486351523840.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 57829261.50841 Elapsed: 21.8704s Converge: 0.975269 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 577621.09281 Elapsed: 20.7190s Converge: 0.181429 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.4972 | 0.0058 | 0.0058 | 0.0216 |  0.0079 |  0.0140 |       0.0035 |    0.0274 |  110.9621 |   0.6642



In [ ]:
data9 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1000,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5230,"F1":0.0023,"MAP":0.0033,"MRR":0.0077,"NCRR":0.0020,"NDCG":0.0050,"Precision":0.0014,"Recall":0.0134,
        "category diversity":compute_avg_diversity(convmf_u1000_v100_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_u1000_v100_k300, mind_, "story")}
print(data9['category diversity'])
print(data9['story diversity'])

0.40605035368034664
0.0005968325217134946


### k=300, lambda_u=100, lambda_v=10

In [ ]:
# Instantiate Conv_mf model
convmf_u100_v10_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=100,
    lambda_v=10,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)
# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_u100_v10_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 233843461.29863 Elapsed: 22.4600s Converge: 23384346129863032562084673744296290832367169493492121468928.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 6050216.49389 Elapsed: 19.3168s Converge: 0.974127 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 54636.83768 Elapsed: 22.4653s Converge: 0.537690 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.7263 | 0.0201 | 0.0230 | 0.0835 |  0.0357 |  0.0555 |       0.0121 |    0.1106 |  107.7720 |   0.6613



In [ ]:
data10 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":100,
        "lambda_v":10,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.7082,"F1":0.0219,"MAP":0.0219,"MRR":0.0698,"NCRR":0.0318,"NDCG":0.0579,"Precision":0.0131,"Recall":0.1276,
        "category diversity":compute_avg_diversity(convmf_u100_v10_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_u100_v10_k300, mind_, "story")}
print(data10['category diversity'])
print(data10['story diversity'])

0.642391776751164
0.005750565245649905


### k=300, lambda_u=100, lambda_v=1

In [ ]:
# Instantiate Conv_mf model
convmf_u100_v1_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=100,
    lambda_v=1,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)
# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_u100_v1_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 23412842.93457 Elapsed: 22.0791s Converge: 2341284293456585304120977202864105903468051149380476993536.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 660015.84804 Elapsed: 21.5881s Converge: 0.971810 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 26158.44091 Elapsed: 22.1146s Converge: 0.312923 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.7290 | 0.0214 | 0.0235 | 0.0651 |  0.0317 |  0.0565 |       0.0128 |    0.1230 |  108.9594 |   1.1197



In [ ]:
data11 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":100,
        "lambda_v":1,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.7276,"F1":0.0201,"MAP":0.0200,"MRR":0.0601,"NCRR":0.0274,"NDCG":0.0513,"Precision":0.0121,"Recall":0.1173,
        "category diversity":compute_avg_diversity(convmf_u100_v1_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_u100_v1_k300, mind_, "story")}
print(data11['category diversity'])
print(data11['story diversity'])

1.2784116139858985
6.106475349412906e-06


### k=300, lambda_u=100, lambda_v=1000

In [ ]:
# Instantiate Conv_mf model
convmf_u100_v1000_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=100,
    lambda_v=1000,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)
# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=mind_ratio_split, models=[convmf_u100_v1000_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 23373693962.00029 Elapsed: 22.5279s Converge: 2337369396200028603656230309475124554385054135414414743961600.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 608225626.85762 Elapsed: 22.0383s Converge: 0.973978 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 9445743.27545 Elapsed: 19.5032s Converge: 0.284397 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/1336 [00:00<?, ?it/s]

Ranking:   0%|          | 0/234 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.0000 | 0.0000 | 0.0000 | 0.5307 | 0.0024 | 0.0040 | 0.0135 |  0.0039 |  0.0056 |       0.0015 |    0.0093 |  108.2669 |   0.7010



In [ ]:
data12 = {"model":"Conv_mf",
        "data":"mind",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":100,
        "lambda_v":1000,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4968,"F1":0.0033,"MAP":0.0030,"MRR":0.0071,"NCRR":0.0019,"NDCG":0.0064,"Precision":0.0020,"Recall":0.0190,
        "category diversity":compute_avg_diversity(convmf_u100_v1000_k300, mind_, "category"),
         "story diversity":compute_avg_diversity(convmf_u100_v1000_k300, mind_, "story")}
print(data12['category diversity'])
print(data12['story diversity'])

0.5460548494459008
0.007949768042644002


In [ ]:
data = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12]
df = pd.DataFrame(columns = data1.keys())
i=0
for d in data:
    df = pd.concat([df, pd.DataFrame(d,columns = list(d.keys()),index=[i])], ignore_index=False)
    i += 1

In [ ]:
df

,model,data,k,n_epochs,cnn_epochs,cnn_bs,cnn_lr,lambda_u,lambda_v,emb_dim,...,AUC,F1,MAP,MRR,NCRR,NDCG,Precision,Recall,category diversity,story diversity
0,Conv_mf,mind,10,10,5,128,0.001,1,100,200,...,0.7076,0.0127,0.0155,0.0386,0.0174,0.0345,0.0074,0.0803,0.292472,0.010355
1,Conv_mf,mind,50,5,5,128,0.001,1,100,200,...,0.6326,0.0075,0.0071,0.0263,0.0085,0.0166,0.0045,0.0375,0.800833,0.006183
2,Conv_mf,mind,100,5,5,128,0.001,1,100,200,...,0.5988,0.0067,0.0096,0.0277,0.0115,0.0186,0.0041,0.0351,0.469408,0.041244
3,Conv_mf,mind,200,5,5,128,0.001,1,100,200,...,0.5240,0.0033,0.0035,0.0094,0.0025,0.0066,0.0020,0.0161,0.182574,0.006561
4,Conv_mf,mind,300,5,5,128,0.001,1,100,200,...,0.5323,0.0028,0.0034,0.0100,0.0025,0.0058,0.0017,0.0148,0.850797,0.013198
5,Conv_mf,mind,400,5,5,128,0.001,1,100,200,...,0.5127,0.0015,0.0029,0.0044,0.0010,0.0041,0.0009,0.0148,1.125643,0.018225
6,Conv_mf,mind,300,5,5,128,0.001,10,100,200,...,0.5495,0.0042,0.0038,0.0100,0.0028,0.0077,0.0025,0.0189,0.716117,0.008274
7,Conv_mf,mind,300,5,5,128,0.001,100,100,200,...,0.5866,0.0061,0.0053,0.0140,0.0048,0.0135,0.0036,0.0367,0.257395,0.001705
8,Conv_mf,mind,300,5,5,128,0.001,1000,100,200,...,0.5230,0.0023,0.0033,0.0077,0.0020,0.0050,0.0014,0.0134,0.406050,0.000597
9,Conv_mf,mind,300,5,5,128,0.001,100,10,200,...,0.7082,0.0219,0.0219,0.0698,0.0318,0.0579,0.0131,0.1276,0.642392,0.005751


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/master-project/conv_mf/convmf_mind.csv',header=-1,index=0)